This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [12]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = ''

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [13]:
# First, import the relevant modules
import requests
import collections


In [14]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned

DATABASE = 'FSE'
DATASET = 'AFX_X'
RETURN_TYPE = 'json'

data = requests.get(f'https://www.quandl.com/api/v3/datasets/{DATABASE}/{DATASET}.{RETURN_TYPE}?start_date=2000-06-06&end_date=2000-06-07&api_key={API_KEY}')



In [15]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure

data.json()

{'dataset': {'id': 10095370,
  'dataset_code': 'AFX_X',
  'database_code': 'FSE',
  'name': 'Carl Zeiss Meditec (AFX_X)',
  'description': 'Stock Prices for Carl Zeiss Meditec (2020-11-02) from the Frankfurt Stock Exchange.<br><br>Trading System: Xetra<br><br>ISIN: DE0005313704',
  'refreshed_at': '2020-12-01T14:48:09.907Z',
  'newest_available_date': '2020-12-01',
  'oldest_available_date': '2000-06-07',
  'column_names': ['Date',
   'Open',
   'High',
   'Low',
   'Close',
   'Change',
   'Traded Volume',
   'Turnover',
   'Last Price of the Day',
   'Daily Traded Units',
   'Daily Turnover'],
  'frequency': 'daily',
  'type': 'Time Series',
  'premium': False,
  'limit': None,
  'transform': None,
  'column_index': None,
  'start_date': '2000-06-07',
  'end_date': '2000-06-07',
  'data': [['2000-06-07',
    None,
    None,
    None,
    38.0,
    None,
    0.0,
    None,
    None,
    None,
    0.0]],
  'collapse': None,
  'order': None,
  'database_id': 6129}}

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [16]:
#Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017

#add Jan. 1st 2017 as the start date and Dec. 31 2017 as the end date to API call
query = requests.get(f'https://www.quandl.com/api/v3/datasets/{DATABASE}/{DATASET}.{RETURN_TYPE}?start_date=2017-01-01&end_date=2017-12-31&api_key={API_KEY}')


In [17]:
#Convert the returned JSON object into a Python dictionary.

#Use the requests library built-in handling of JSON objects
jdata = query.json()

#Since there is only one key in the top level, 
#store the value as a variable to avoid repeating this statement
jdataset = jdata['dataset']

In [18]:
#Calculate what the highest and lowest opening prices were for the stock in this period.

#Store the number of dates with data
num_entries = len(jdataset['data'])

#record the column within each jdataset['data'] entry corresponding to Open
open_column = jdataset['column_names'].index('Open')

#generate list of opening prices for each date in data
openings = [jdataset['data'][entry][open_column] for entry in range(0,num_entries)]

#remove missing entries
openings = [x for x in openings if x is not None]

#store the minimum and maximum opening prices
min_open = min(openings)
max_open = max(openings)

#print the results
print(f'Lowest opening price: {min_open}')
print(f'Highest opening price: {max_open}')

Lowest opening price: 34.0
Highest opening price: 53.11


In [20]:
#What was the largest change in any one day (based on High and Low price)?

#record the columns within each jdataset['data'] entry corresponding to High and Low
high_column = jdataset['column_names'].index('High')
low_column = jdataset['column_names'].index('Low')

#calculate the daily change in price by subtracting each day's high and low price
daily_change = [jdataset['data'][entry][high_column] - jdataset['data'][entry][low_column]
               for entry in range(0,num_entries)]

#find the maximum daily change and print the result
max_change = round(max(daily_change), 2)

print(f'Largest change in one day: {max_change}')

Largest change in one day: 2.81


In [21]:
#What was the largest change between any two days (based on Closing Price)?

#record the columns within each jdataset['data'] entry corresponding to Close
close_column = jdataset['column_names'].index('Close')

#generate list of closing prices for each date in data
closings = [jdataset['data'][entry][close_column] for entry in range(0,num_entries)]

#calculate the maximum change between two days by subtracting the min value from the max value 
max_daychange = round(max(closings) - min(closings), 2)

print(f'Largest change between any two days: {max_daychange}')

Largest change between any two days: 19.03


In [25]:
#What was the average daily trading volume during this year?

#record the columns within each jdataset['data'] entry corresponding to Traded Volume
volume_column = jdataset['column_names'].index('Traded Volume')

#generate list of traded volume for each date in data
volume = [jdataset['data'][entry][volume_column] for entry in range(0,num_entries)]

#calculate the mean volume and print the results
mean_volume = int(sum(volume)/len(volume))
print(f'Average daily trading volume in 2017: {mean_volume}')


Average daily trading volume in 2017: 89124


In [28]:
#(Optional) What was the median trading volume during this year?

#write function that inputs a list
def calcMedian(list):
    
    #sort the list before finding median
    sort_list = sorted(list)
    
    #test if the list is odd in length
    if len(sort_list) % 2 > 0:
        
        #if so, find the middle index with floor division
        middle = len(sort_list)//2
        median = sort_list[middle]
    
    #if the list is even in length, take the average of the two middle numbers
    else:
        middle = int(len(sort_list)/2) - 1
        median = sum(sort_list[middle:(middle+2)]) / 2
    
    #return the result
    return(median)

#calculate the median of the daily traded volume and print the results
median_volume = int(calcMedian(volume))
print(f'Median trading volume in 2017: {median_volume}')

Median trading volume in 2017: 76286
